### OpenAI Natural Language to SQL Evaluation -- sqlite
#### Utilizing token counting informmation from https://platform.openai.com/docs/api-reference/making-requests

In [1]:
import os
import getpass
from dotenv import load_dotenv, dotenv_values
import pandas as pd
import openai

from IPython.display import display, Markdown, Latex, HTML, JSON
import sqlite3
from sqlite3 import Error

import tiktoken

import langchain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from cmd import PROMPT
import os
from pyexpat.errors import messages
import openai

In [33]:
class GenAI_NL2SQL():
    def __init__(self, OPENAI_API_KEY, Model, Encoding_Base, Max_Tokens, Temperature, Token_Cost):
        self._Model = Model
        self._Encoding_Base = Encoding_Base
        self._Max_Tokens = Max_Tokens
        self._Temperature = Temperature
        self._Token_Cost = Token_Cost
        self._OpenAI_API_Key = OPENAI_API_KEY

    def Send_OpenAI_API_Key(self):
        openai.api_key = self._OpenAI_API_Key
        return 0

    # count the number of input tokens in prompt
    def Num_Tokens_From_String(self, Prompt: str, Verbose=False) -> int:
        """Returns the number of tokens in a text string."""
        encoding = tiktoken.get_encoding(self._Encoding_Base)
        num_tokens = len(encoding.encode(Prompt))
        if Verbose:
            print(f'number of tokens {num_tokens}')
        return num_tokens
    
    # Extimate prompt cost per GPT model
    def Prompt_Cost(self, Prompt):
        Cost = Token_Cost[self._Model]  # cost per 1K tokens
        Input_Cost = Cost['Input']*self.Num_Tokens_From_String(Prompt)
        return(Input_Cost, self.Num_Tokens_From_String(Prompt, Verbose=False))
    
    def OpenAI_Usage_Cost(self, Response):
        Cost = self._Token_Cost[self._Model]  # cost per 1K tokens
        Total_Cost = Cost['Input']*Response['usage']['prompt_tokens'] + \
            Cost['Output']*Response['usage']['completion_tokens']
        return(Total_Cost, Response['usage']['total_tokens'])
    
    def Prompt_Question(self, _Prompt_Template_, Inputs):
        """
        """
        for i,j in Inputs.items():
            Prompt = _Prompt_Template_.replace(i,j)
        return Prompt

    def OpenAI_Completion(self, Prompt):
        Response = openai.Completion.create(
            model=self._Model,
            prompt=Prompt,
            max_tokens=self._Max_Tokens,
            temperature=self._Temperature
        )
        return(Response)

    def OpenAI_Text_Extraction(self, Response, Type='SQL'):
        if Type == 'SQL':
            Txt = Response['choices'][0]['text']
        else:
            print(f'Type: {Type} is Unsupported ')
            txt = ''
        return(Txt)


    def execute_query(self, conn, query=None):
        """
        """
        cur = conn.cursor()
        cur.execute(query)
        #cur.execute("SELECT * FROM employee limit 5")
    
        rows = cur.fetchall()
    
        for row in rows:
            print(row)

    def run_query(self, Conn=None, DB=None, DBFile = None, Query=None):
        """
        """
        if Conn:
            print('here ',Conn)
            try:
                return(pd.DataFrame(pd.read_sql(Query, Conn)))
            except Error as e:
                print(e)
                
        else:
            if DB == 'sqlite':
                try:
                    Conn = sqlite3.connect(DBFile)
                except Error as e:
                    print(e)
            try:
                return(pd.DataFrame(pd.read_sql(Query, Conn)))
            except Error as e:
                print(e)
    
        return 0


    def LangChain_Initiate_LLM(self, Model='OpenAI'):
        if Model=='OpenAI':
            self._LLM = OpenAI(temperature=self._Temperature, model_name=self._Model, \
                max_tokens=self._Max_Tokens, openai_api_key=self._OpenAI_API_Key)
            return 0
        else:
            print('Model Unsupported')
            return -1
            
    # Langchain Completion
    def LangChainCompletion(self, Prompt, Input):
        chain = LLMChain(llm=self._LLM, prompt=Prompt)
        return chain.run(Input) 

#### Token Accounting

In [70]:
# Input/Output costs per Model
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
#openai.api_key = OPENAI_API_KEY

# Davinci--3 has been replaced by gpt-3.5-turbo-instruct
Model =  "gpt-3.5-turbo-instruct"
Encoding_Base="cl100k_base"
Max_Tokens = 250
Temperature=0
Token_Cost = {"gpt-3.5-turbo-instruct":{"Input":0.0015/1000,"Output":0.002/1000}}
GPT3 = GenAI_NL2SQL(OPENAI_API_KEY, Model, Encoding_Base, Max_Tokens, Temperature, Token_Cost)
GPT3.Send_OpenAI_API_Key()
DB = 'sqlite'
db_file =  r"FakeBank.db"

In [9]:
# Encode/Decode
# encode text
encoding = tiktoken.get_encoding(Encoding_Base)

encode_txt = encoding.encode("tiktoken is great!")
print(f'encodings:  {encode_txt}')

# decode tokens to numbers 
decode_txt = encoding.decode(encode_txt)
print(f'dectings:  {decode_txt}')

encodings:  [83, 1609, 5963, 374, 2294, 0]
dectings:  tiktoken is great!


#### DB Interface

In [68]:
_Prompt_Template_ = """You are a SQLite expert. 
Given an input question, first create and return a syntactically correct sql query.
Unless the user specifies a specific number of rows to obtain in the question,
the query should return for at most 10 results using the LIMIT clause as per SQLite. 

You may order the results to return the most informative data in the database.
Never query for all columns from a table. 
You must query only the columns that are needed to answer the question. 

Pay attention to use only the column names you can see in the tables below. 
Be careful to exclude columns that do not exist in the database.
Pay attention to which column is in which table.

Unless specified in the question, insert "relationship.relationship_type ='Primary'" in the where
clause when joining on the relationship table.
When asked about account types, use the "type" field in the product table. 
When asked about a client id, use the "id" field in the client table.
When asked about a clients who share a household, join the "client" table with "HH" table.
When asked for the full or complete address, return "address, city, state and zip" from the address table.
Never use First_Name and Last_Name when querying the client table.

When asked for portfolio summary, return account type, number of open accounts, number of closed accounts, total balances,
month to date average balance

Only use the following tables:

CREATE TABLE client
(
	ID integer PRIMARY KEY,
    Client_ID integer not null,
    HH_ID integer not null,
    Person_Org text not null,
    Name text not null,
    FirstName text ,
    LastName text ,
    SSN text,
    Address_ID integer not null,
    Banker_ID integer not null,
    Employer_ID integer not null,
    Open_Dt text not null,
    Start_Dt text not null,
    End_Dt text not null

)
Create table HH
(
    ID integer PRIMARY KEY,
    Person_Org text not null,
    Name text not null,
    FirstName text ,
    LastName text ,
    Address_ID integer not null,
    Banker_ID integer not null,
    Open_Dt text not null,
    FOREIGN KEY(Address_ID) REFERENCES address(ID),
    FOREIGN KEY(Banker_ID) REFERENCES banker(ID)
)

create table address
(
    ID integer primary key,
    Address text,
    City text,
    State text,
    Zip text
)

create table  account
(
    ID integer primary key,
    acct_nbr text not null,
    Product_ID integer not null,
    Banker_ID integer not null,
    Status text not null,
    Open_Dt text not null,
    Close_Dt text,
    FOREIGN KEY(Product_ID) REFERENCES Product(ID),
    FOREIGN KEY(Banker_ID) REFERENCES banker(ID)   
)

create table relationship
(
    ID integer primary key,
    Account_ID integer not null,
    Client_ID integer not null,
    Relationship_Type text not null,
    FOREIGN KEY(Account_ID) REFERENCES account(ID),
    FOREIGN KEY(Client_ID) REFERENCES client(ID)
)

create table  balance
(
    ID integer primary key,
    Account_ID integer not null,
    Balance real not null,
    Balance_Dt text not null,
    FOREIGN KEY(Account_ID) REFERENCES account(ID)
)

create table employer
(
    ID integer primary key,
    Name text,
    Address_ID integer not null,
    Client_ID integer,
    Industry text,
    Start_Dt text not null,
    End_Dt text not null,
    Is_Current integer not null,
    FOREIGN KEY(Address_ID) REFERENCES address(ID)
)

USER
{Question}
"""



In [13]:
# Add Question to Prompt Template
Question = 'Which clients live in household id = 3? Return the client id, client name and the full client address.'
Input_Vars = {'{Question}':Question}

# Input
Prompt = GPT3.Prompt_Question(_Prompt_Template_,Input_Vars)
Cost, Tokens_Used  = GPT3.Prompt_Cost(Prompt)
print('Input')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')


Response = GPT3.OpenAI_Completion(Prompt)
Cost, Tokens_Used  = GPT3.OpenAI_Usage_Cost(Response)
print('Output')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')
print()

Query = GPT3.OpenAI_Text_Extraction(Response, Type='SQL')
GPT3.run_query(DB=DB,DBFile = db_file, Query = Query)


Input
Total Cost: 0.001 Tokens Used 619
Output
Total Cost: 0.001 Tokens Used 677



,Client_ID,Name,Address,City,State,Zip
0,4,Mary Spock,1023 Arcadia Blvd,La Jolla,CA,92037
1,7,George Spock,1023 Arcadia Blvd,La Jolla,CA,92037


In [31]:
rtn = GPT3.LangChain_Initiate_LLM(Model='OpenAI')
Prompt = PromptTemplate(
#    input_variables=["input", "table_info","dialect"], template=_DEFAULT_TEMPLATE
    input_variables=["Question"], template=_Prompt_Template_ 
)

rtn = GPT3.LangChainCompletion(Prompt=Prompt, Input=Question)
print(rtn)


SELECT client.Client_ID, client.Name, address.Address, address.City, address.State, address.Zip 
FROM client 
INNER JOIN address ON client.Address_ID = address.ID 
INNER JOIN HH ON client.HH_ID = HH.ID 
WHERE HH.ID = 3 
LIMIT 10;


In [42]:
# Add Question to Prompt Template
Question = 'What are Mary Spock Account Balances. Return date, account type, account number, and balance'
Input_Vars = {'{Question}':Question}

# Input
Prompt = GPT3.Prompt_Question(_Prompt_Template_,Input_Vars)
Cost, Tokens_Used  = GPT3.Prompt_Cost(Prompt)
print('Input')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')


Response = GPT3.OpenAI_Completion(Prompt)
Cost, Tokens_Used  = GPT3.OpenAI_Usage_Cost(Response)
print('Output')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')
print()

Query = GPT3.OpenAI_Text_Extraction(Response, Type='SQL')
print(Query)
GPT3.run_query(DB=DB,DBFile = db_file, Query = Query)


Input
Total Cost: 0.001 Tokens Used 715
Output
Total Cost: 0.001 Tokens Used 810


SELECT balance.Balance_Dt, product.type, account.acct_nbr, balance.Balance
FROM balance
INNER JOIN account ON balance.Account_ID = account.ID
INNER JOIN product ON account.Product_ID = product.ID
INNER JOIN relationship ON account.ID = relationship.Account_ID
INNER JOIN client ON relationship.Client_ID = client.ID
WHERE client.FirstName = 'Mary' AND client.LastName = 'Spock'
AND relationship.Relationship_Type = 'Primary'
LIMIT 10;


,Balance_Dt,Type,acct_nbr,Balance
0,2023-10-02 00:00:00,Checking,C766,1538.222668
1,2023-10-02 00:00:00,Checking,C810,189.278709
2,2023-10-02 00:00:00,Checking,C782,2264.491239
3,2023-10-02 00:00:00,Checking,C922,7075.588054
4,2023-10-02 00:00:00,Checking,C1881,1775.081527
5,2023-10-02 00:00:00,Saving,CD343,7583.868611
6,2023-10-02 00:00:00,Saving,S922,3849.894714
7,2023-10-02 00:00:00,Loan,SF958,11483.760918


In [39]:
# Add Question to Prompt Template
Question = 'What are Mary Spock Account Balances. Return date, account type, sum(balance) called Total'
Input_Vars = {'{Question}':Question}

# Input
Prompt = GPT3.Prompt_Question(_Prompt_Template_,Input_Vars)
Cost, Tokens_Used  = GPT3.Prompt_Cost(Prompt)
print('Input')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')


Response = GPT3.OpenAI_Completion(Prompt)
Cost, Tokens_Used  = GPT3.OpenAI_Usage_Cost(Response)
print('Output')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')
print()

Query = GPT3.OpenAI_Text_Extraction(Response, Type='SQL')
print(Query)
GPT3.run_query(DB=DB,DBFile = db_file, Query = Query)

Input
Total Cost: 0.001 Tokens Used 617
Output
Total Cost: 0.001 Tokens Used 727


SELECT balance.Balance_Dt AS date, product.type AS account_type, SUM(balance.Balance) AS Total
FROM balance
INNER JOIN account ON balance.Account_ID = account.ID
INNER JOIN product ON account.Product_ID = product.ID
INNER JOIN relationship ON account.ID = relationship.Account_ID
INNER JOIN client ON relationship.Client_ID = client.ID
WHERE client.FirstName = 'Mary' AND client.LastName = 'Spock' AND relationship.Relationship_Type = 'Primary'
GROUP BY balance.Balance_Dt, product.type
LIMIT 10;


,date,account_type,Total
0,2023-10-02 00:00:00,Checking,12842.662197
1,2023-10-02 00:00:00,Loan,11483.760918
2,2023-10-02 00:00:00,Saving,11433.763325


In [71]:
# Add Question to Prompt Template
Question = "How many of Mary Spock's, ssn = 456-78-9012, accounts are open and closed by account type?  \
Return account type, total number of open and total number of closed accounts"
Input_Vars = {'{Question}':Question}

# Input
Prompt = GPT3.Prompt_Question(_Prompt_Template_,Input_Vars)
Cost, Tokens_Used  = GPT3.Prompt_Cost(Prompt)
print('Input')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')


Response = GPT3.OpenAI_Completion(Prompt)
Cost, Tokens_Used  = GPT3.OpenAI_Usage_Cost(Response)
print('Output')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')
print()

Query = GPT3.OpenAI_Text_Extraction(Response, Type='SQL')
print(Query)
GPT3.run_query(DB=DB,DBFile = db_file, Query = Query)

Input
Total Cost: 0.001 Tokens Used 783
Output
Total Cost: 0.001 Tokens Used 894


SELECT type, COUNT(CASE WHEN status = 'Open' THEN 1 END) AS 'Number of Open Accounts', COUNT(CASE WHEN status = 'Closed' THEN 1 END) AS 'Number of Closed Accounts'
FROM account
INNER JOIN product ON account.product_id = product.id
INNER JOIN relationship ON account.id = relationship.account_id
INNER JOIN client ON relationship.client_id = client.id
WHERE client.ssn = '456-78-9012' AND relationship.relationship_type = 'Primary'
GROUP BY type
LIMIT 10;


,Type,Number of Open Accounts,Number of Closed Accounts
0,Checking,3,2
1,Loan,1,0
2,Saving,1,1


In [73]:
# Add Question to Prompt Template
Question = "What is Mary Spock's, ssn = 456-78-9012, portfolio summary?"
Input_Vars = {'{Question}':Question}

# Input
Prompt = GPT3.Prompt_Question(_Prompt_Template_,Input_Vars)
Cost, Tokens_Used  = GPT3.Prompt_Cost(Prompt)
print('Input')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')


Response = GPT3.OpenAI_Completion(Prompt)
Cost, Tokens_Used  = GPT3.OpenAI_Usage_Cost(Response)
print('Output')
print(f'Total Cost: {round(Cost,3)} Tokens Used {Tokens_Used}')
print()

Query = GPT3.OpenAI_Text_Extraction(Response, Type='SQL')
print(Query)
GPT3.run_query(DB=DB,DBFile = db_file, Query = Query)

Input
Total Cost: 0.001 Tokens Used 760
Output
Total Cost: 0.001 Tokens Used 928


SELECT product.type AS "Account Type", 
COUNT(CASE WHEN account.status = 'Open' THEN 1 END) AS "Number of Open Accounts", 
COUNT(CASE WHEN account.status = 'Closed' THEN 1 END) AS "Number of Closed Accounts", 
SUM(balance.balance) AS "Total Balances", 
AVG(balance.balance) AS "Month to Date Average Balance" 
FROM client 
INNER JOIN relationship ON client.id = relationship.client_id 
INNER JOIN account ON relationship.account_id = account.id 
INNER JOIN product ON account.product_id = product.id 
INNER JOIN balance ON account.id = balance.account_id 
WHERE client.name = 'Mary Spock' AND client.ssn = '456-78-9012' AND relationship.relationship_type = 'Primary' 
GROUP BY product.type 
LIMIT 10;


,Account Type,Number of Open Accounts,Number of Closed Accounts,Total Balances,Month to Date Average Balance
0,Checking,126,84,82831.079855,394.433714
1,Loan,42,0,203036.976354,4834.213723
2,Saving,42,42,134085.493661,1596.255877


In [ ]:
__PROMT_TEMPLATE__ = """
You are a SQLite expert. 
Given an input question, first create a syntactically correct SQLite query to run, 
then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, 
query for at most 10 results using the LIMIT clause as per SQLite. 

You can order the results to return the most informative data in the database.
Never query for all columns from a table. 
You must query only the columns that are needed to answer the question. 

Pay attention to use only the column names you can see in the tables below. 
Be careful to not query for columns that do not exist. 
Also, pay attention to which column is in which table.

When using the relationship table, insert relationship.relationshiptype ='Primary'
When asked about account types, use the type field in the product table. 
When asked about a client id, use the id field in the client table.
When asked about a clients who share a household, join clients with HH table.
When asked for the full or complete address, return address, city, state and zip

Only use the following tables:

CREATE TABLE client
(
	ID integer PRIMARY KEY,
    Client_ID integer not null,
    HH_ID integer not null,
    Person_Org text not null,
    Name text not null,
    FirstName text ,
    LastName text ,
    Address_ID integer not null,
    Banker_ID integer not null,
    Employer_ID integer not null,
    Open_Dt text not null,
    Start_Dt text not null,
    End_Dt text not null

)
create table address
(
    ID integer primary key,
    Address text,
    City text,
    State text,
    Zip text
)

create table account
(
    ID integer primary key,
    acct_nbr text not null,
    Product_ID integer not null,
    Banker_ID integer not null,
    FOREIGN KEY(Product_ID) REFERENCES Product(ID),
    FOREIGN KEY(Banker_ID) REFERENCES banker(ID)   
)

create table relationship
(
    ID integer primary key,
    Account_ID integer not null,
    Client_ID integer not null,
    Relationship_Type text not null,
    FOREIGN KEY(Account_ID) REFERENCES account(ID),
    FOREIGN KEY(Client_ID) REFERENCES client(ID)
)

create table balance
(
    ID integer primary key,
    Account_ID integer not null,
    Balance real not null,
    Balance_Dt text not null,
    FOREIGN KEY(Account_ID) REFERENCES account(ID)
)

create table  employer
(
    ID integer primary key,
    Name text,
    Address_ID integer not null,
    Client_ID integer,
    Industry text,
    Start_Dt text not null,
    End_Dt text not null,
    Is_Current integer not null,
    FOREIGN KEY(Address_ID) REFERENCES address(ID)
)

question: {input}
"""
PROMPT = PromptTemplate(
    input_variables=["input"], template=__PROMT_TEMPLATE__ 
)

In [ ]:
PROMPT

In [ ]:
langchain.debug = True
question = "Who lives in the same household as Mary Spock? Return the client id, household id and client name"
llm = OpenAI(temperature=0.0, model=Model, max_tokens=150)
chain = LLMChain(llm=llm, prompt=PROMPT)
text = chain.run(question)
print(text)

In [ ]:
text = query
print(execute_query(conn, query))